In [1]:
%reload_ext autoreload
%autoreload 2

%reload_ext autoreload
%autoreload 2

import os
import sys
import shutil
sys.path.append('..') 

import numpy as np
import matplotlib.pyplot as plt
import h5py

import plot_utils as pu
from code.bin_newer.CFD_solver import Fluid2D
from utils import WaveIcs

In [26]:
N = 100
lim = 1
AD = 1e-1
k_x, k_y = 1 * 2*np.pi, 0 * 2*np.pi
D0 = 1
P0 = 1

Ics = WaveIcs(N, lim)
Ics.set_perturbation(AD, k_x, k_y, D0=D0, P0=P0)

output_folder = f'../results/waves/test/'
T = 5
CFL_factor = 1

U = Ics.U
V = Ics.V

Fluid = Fluid2D(Ics.x, Ics.y, Ics.D, U, V, Ics.P, CFL_factor=CFL_factor)
Fluid.set_output_folder(output_folder)
Fluid.evaluate(T)

Compute time: 5.65e-01 s
Number of snapshots: 49


In [28]:
n_plots = None
fields = ['Pressure', 'Density', 'Velocity X', 'Velocity Y']

save_folder = output_folder + 'frames/'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
else:
    shutil.rmtree(save_folder)
    os.makedirs(save_folder)


with h5py.File(Fluid.output_data_path, 'r') as file:

    M = file['Header'].attrs['N']
    x = file['Header'].attrs['X']
    y = file['Header'].attrs['Y']

    X, Y = np.meshgrid(x, y)


    if n_plots is None:
        n_plots = M
    idxs = np.linspace(0, M-1, n_plots, dtype=int)

    cmap = 'coolwarm'
    Fig = pu.Figure(subplots=(2,2), fig_size=540, ratio=1, theme='default', grid=False)
    axs = Fig.axes_flat
    fs = Fig.fs
    ts = Fig.ts

    idx = 0
    min_max = {}
    for i, field_name in enumerate(fields):
        for k in range(M):
            step_group = file[f"{k:03d}"]
            field = step_group[field_name][()]
            if k == 0:
                max_field = np.max(field)
                min_field = np.min(field)
            else:
                max_field = np.max([max_field, np.max(field)])
                min_field = np.min([min_field, np.min(field)])

            min_max[field_name] = (min_field, max_field)

        axs[i].set_aspect("equal", "box")
        axs[i].text(0.5, 1.05, field_name, ha='center', va='center', transform=axs[i].transAxes, fontsize=fs*ts, color='k')

    for i, idx in enumerate(idxs):
        ims = []
        for j, field_name in enumerate(fields):

            field = np.array(file[f'{idx:03d}'][field_name])

            extent = [x[0], x[-1], y[0], y[-1]]
            im = axs[j].pcolormesh(X, Y, field, vmin=min_max[field_name][0], vmax=min_max[field_name][1], cmap=cmap)
            ims.append(im)


        image_path = save_folder + f'frame_{i:04d}.jpg'
        Fig.save(image_path)

        plt.close()
        for im in ims:
            im.remove()

        print(f"Saved frame {i+1}/{n_plots}", end='\r')

save_folder = output_folder + 'frames/'
pu.frames_to_mp4(save_folder, fps=20)

save_path = output_folder + 'conserved_quantites.jpg' 
pu.plot_conserved(Fluid.output_data_path, save_path)